In [21]:
# Imports local
import sys
sys.path.insert(0, "../")

In [ ]:
#Imports from pip
!pip install git+https://github.com/esgantivar/qmc.git@tn -q

In [23]:
import tensorflow as tf
import numpy as np
from qmc.tn.models import MPSClassifier
from qmc.tn.features_map import binary_map

In [28]:
#Extract the MNIST Images and preprocess
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# apply feature map
x_train = binary_map(x_train)
x_test = binary_map(x_test)
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# Y one hot
# y_train = y_train[:, np.newaxis]
# y_test = y_test[:, np.newaxis]
x_test.shape, y_train.shape, x_train.shape, y_test.shape

((10000, 784, 2), (60000, 10), (60000, 784, 2), (10000, 10))

In [25]:
model = MPSClassifier(d_bond=2)

In [29]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [30]:
history = model.fit(
    x_train,
    y_train,
    epochs=2,
    verbose=1,
    batch_size=1,
    validation_data=(x_test, y_test)
)

Epoch 1/2
1875/1875 [==============================] - 24s 12ms/step - loss: 1.3482 - accuracy: 0.5188 - val_loss: 0.9038 - val_accuracy: 0.7225
Epoch 2/2
1875/1875 [==============================] - 20s 11ms/step - loss: 0.9365 - accuracy: 0.7055 - val_loss: 1.1678 - val_accuracy: 0.6021


In [32]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.1678 - accuracy: 0.6021


[1.1677708625793457, 0.6021000146865845]